# PySpark Dataframes
For each instruction, type the appropriate code into the cell below the instruction. Then, run the code by pressing the `Run` button above.

1. Import the SparkSession class:

```
from pyspark.sql import SparkSession
```

In [1]:
from pyspark.sql import SparkSession

2. Use this class to instiate a Spark session:

```
spark = SparkSession \
    .builder \
    .appName("My First PySpark App") \
    .getOrCreate()
```

In [4]:
spark = SparkSession \
    .builder \
    .appName("My First PySpark App") \
    .getOrCreate()

3. Take a look at the session object:

```
spark
```

In [6]:
spark

4. Read the contents of a csv file into a Dataframe named 'accounts':

```
accounts = spark.read.option('header', 'true').csv('./data/accounts.csv')
```

In [8]:
accounts = spark.read.option('header', 'true').csv('./data/accounts.csv')

5. Take a look at the Dataframe's schema:

```
accounts.printSchema()
```

In [9]:
accounts.printSchema()

root
 |-- account_number: string (nullable = true)
 |-- aba: string (nullable = true)
 |-- bic: string (nullable = true)
 |-- opened: string (nullable = true)
 |-- balance: string (nullable = true)



6. Read the contents of a parquet file into a variable:
```
transactions = spark.read.option('header', True).parquet('./data/transactions.parquet')
```

In [11]:
transactions = spark.read.option('header', True).parquet('./data/transactions.parquet')

7. See how many rows are in the new Dataframe:
```
transactions.count()
```

In [12]:
transactions.count()

1000000

8. Make a new Dataframe by grouping the transactions by account number and summing the groups. This will combine the transactions per account:

```
account_transactions = transactions.groupby('account_number').sum()
```

In [18]:
account_transactions = transactions.groupby('account_number').sum()

9. Combine the accounts with the summed transaction values:

```
with_sum = accounts.join(account_transactions, 'account_number', 'inner')
```

In [19]:
with_sum = accounts.join(account_transactions, 'account_number', 'inner')

10. Get the current balance per account by summing the transaction sums with the initial account balance:

```
accounts = with_sum.withColumn('new_balance', sum([with_sum.balance, with_sum['sum(amount)']]))
```

In [20]:
accounts = with_sum.withColumn('new_balance', sum([with_sum.balance, with_sum['sum(amount)']]))

11. Get accounts with negative current balances:

```
neg_balance = accounts.filter(accounts.new_balance < 0)
```

In [21]:
neg_balance = accounts.filter(accounts.new_balance < 0)

12. Read client data from a json file:

```
clients = spark.read.json('./data/clients.json')
```

In [22]:
clients = spark.read.json('./data/clients.json')

13. Get the clients with a negative balance:

```
clients = clients.join(neg_balance, 'account_number', 'inner')
```

In [23]:
clients = clients.join(neg_balance, 'account_number', 'inner')

14. Look at the top five clients with negative balances:

```
clients.select(['first_name', 'last_name', 'account_number', 'new_balance']).show(5)
```

In [26]:
clients.select(['first_name', 'last_name', 'account_number', 'new_balance']).show(5)

+----------+---------+------------------+-----------+
|first_name|last_name|    account_number|new_balance|
+----------+---------+------------------+-----------+
|    Meagan| Sandoval|JMTP45763117901514|   -27573.0|
|  Michelle|   Knight|RBUE05237750254383|  -103459.0|
|      Paul|   Massey|RJMY57096756148587|   -58329.0|
|  Michelle|    Perez|ZYMB62177146259441|   -55431.0|
|     David|    Green|LRTH65732611614073|  -103831.0|
+----------+---------+------------------+-----------+
only showing top 5 rows

